<a href="https://colab.research.google.com/github/sreedeepack/Q-Exchange/blob/dev/IR%20workbench.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from collections import defaultdict
from indexing import Preprocessor,Indexer
import re

In [2]:
# for testing
doc_map = defaultdict()

In [3]:
import jsonlines

def clean_str(text) :
        text = (text.encode('ascii', 'ignore')).decode("utf-8")
        text = re.sub("&.*?;", "", text)
        text = re.sub("[\]\|\[\@\,\$\%\*\&\\\(\)\":]", "", text)
        text = re.sub("-", " ", text)
        text = re.sub("\.+", "", text)
        text = re.sub("^\s+","" ,text)
        text = re.sub("\.+", "", text)
        text = text.lower()
        return text

def document_generator(file):
    with jsonlines.open(file) as reader:
        for doc_id, obj in enumerate(reader):
            item = {'doc_id': doc_id, 'url': obj['url'], 'title': clean_str(obj['title']), 'desc': clean_str(obj['desc'])}
            yield item

In [4]:
p = Preprocessor()
indexer = Indexer(p)

In [5]:
inverted = {}
for doc in document_generator("crawlers/stack/data/solr.jsonl"):
    doc_id = doc['doc_id']
    # index both title and description
    text = doc['title'] + " " +doc['desc']
    doc_map[doc_id] = (doc['url'], text)
    doc_index = indexer.inverted_index(text)
    indexer.inverted_index_add(inverted,doc_id=doc_id,doc_index=doc_index)

In [6]:
# Print Inverted-Index (3 rows)
i = 0
for word, doc_locations in inverted.items():
    print(word, doc_locations)

    i += 1
    if i > 3:
        break

instal {0: [7, 72, 130, 261, 532, 816, 1791, 2126], 115: [1911, 2734], 188: [831, 1578], 829: [6270, 11787], 1396: [9955], 1408: [1402, 3273], 1414: [6401], 1431: [6157, 15786], 1436: [469, 1907, 2443, 3686, 3790, 4362, 4376, 4419, 4702, 4779, 4862], 1440: [2189, 4407], 1444: [26, 100, 559, 668, 812, 870, 904, 1051, 1146, 1262, 1373, 1394, 2011, 2112, 2309, 2955, 3009, 3307, 3413, 3509, 3556, 3603, 3910, 3992, 4063, 4138, 4297, 4363, 4913, 5103, 5381], 1452: [6377], 1476: [974, 1388, 1589, 1782, 1847, 1902, 2698, 2883, 3024, 3656, 4117, 4151, 4208, 4299, 4795, 5154, 5233, 5554, 5767, 5822, 8222, 8429, 8468, 8830, 9275, 10102, 10208, 10682], 1487: [6262, 9466], 1490: [379, 2707, 4117, 4159, 4476, 5727], 1494: [468, 570, 620, 896, 2139, 2187, 2960, 3920, 4285, 5417, 5511], 1497: [71, 155, 270, 368, 979, 1414, 1543, 2274, 2297], 1499: [42, 1100], 1501: [728, 2510, 4221, 4536, 5115, 5164, 6005, 7975, 8905], 1504: [31, 413, 476, 1106, 1218, 1602, 1939, 2394], 1506: [240, 1612, 3965, 4393, 4

Sample querying

In [10]:
# Search something and print results
queries = ['dolby', 'vim emacs', 'github week']

for query in queries:

    tokenized_query = ' '.join(p.tokenize_string(query))
    result_docs = indexer.search(inverted, tokenized_query)
    print(f"Search for '{query}': doc_ids={result_docs}")

    for _, word in p.word_index(tokenized_query):

        def extract_text(doc_id, position):
            return doc_map[doc_id][1][position:position+50].replace("\n", ' ')

        for doc_id in result_docs:
            for position in inverted[word][doc_id]:
                print(
                    f"\t - {extract_text(doc_id, position)}..."
                    f"\n\t -->{doc_map[doc_id][0]}"
                    f"\n"
                )
        print("\n")

Search for 'dolby': doc_ids={11074}
	 - dolby digital expires at midnight ...
	 -->/r/programming/comments/60b7kv/the_last_patent_on_ac3_dolby_digital_expires_at/



Search for 'vim emacs': doc_ids={3942, 2058, 2672, 3315, 3732, 4221, 1431, 2587, 2877}
	 - vim running inside gnome terminal showing the diff...
	 -->https://askubuntu.com/questions/48299/what-ides-are-available-for-ubuntu

	 - vim splits or extra tabs you like to install it in...
	 -->https://askubuntu.com/questions/48299/what-ides-are-available-for-ubuntu

	 - vim emacs nano gedit kate to name a few enable res...
	 -->https://askubuntu.com/questions/68918/how-do-i-restrict-my-kids-computing-time

	 - vim or emacs to write c code  just try this on you...
	 -->https://askubuntu.com/questions/61408/what-is-a-command-to-compile-and-run-c-programs

	 - vim is amazing! vim is a highly configurable text ...
	 -->https://askubuntu.com/questions/10998/what-developer-text-editors-are-available-for-ubuntu

	 - vim is a highly confi